# SHACL tutorial

This tutorial is based on the youtube webinar of Holger Knublauch  [SHACL Tutorial](https://www.youtube.com/watch?v=ccs-KhnWR1U)
    
For SKOS, see also
 [SKOS Tutorial](https://www.youtube.com/watch?v=3Q4v6vzj3Qk) by [DataFlair](https://www.youtube.com/channel/UCs6nmQZQ6pO1qG0e2fV6G7g).

and
 [SKOS Primer](https://www.w3.org/TR/skos-primer/) and the [SKOS Reference](https://www.w3.org/TR/skos-reference/).

In [2]:
import rdflib

import matplotlib
import matplotlib.pyplot as plt

from pyshacl import Validator
from rdflib import Graph
# inline matplotlib plotting in jupyter notebook

%matplotlib inline

In [27]:
# create a simple SKOS RDF graph (turtle format)

simple_skos_ttl_graph = """
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix ex: <http://example.org/> .
@prefix shdemo: <http://w3id.org/oso/shacl-skos-demo-ontology/> .

<http://w3id.org/oso/shacl-skos-demo-ontology> rdf:type owl:Ontology .

shdemo:City
    a owl:Class ;
    a sh:NodeShape ;
    rdfs:label "City"@en ;
    rdfs:subClassOf skos:Concept;
    sh:property shdemo:City-Population;
    .
# property shapes for City
shdemo:City-Population
    a sh:PropertyShape ;
    sh:path shdemo:population ;
    sh:datatype xsd:integer ;
    sh:minCount 1 ;
    sh:maxCount 1 ;
    sh:name "population"@en ;
    .

shdemo:Country
    a owl:Class ;
    a sh:NodeShape ;
    rdfs:label "Country"@en ;
    rdfs:subClassOf skos:Concept;
    sh:property shdemo:Country-Population;
    .

# property shapes for Country
shdemo:Country-isoCode
    a sh:PropertyShape ;
    sh:path shdemo:isoCode ;
    sh:datatype xsd:string ;
    sh:minCount 1 ;
    sh:maxCount 1 ;
    sh:name "ISO Code"@en ;
   
    .    

"""


In [28]:
g = rdflib.Graph()
g.parse(data=simple_skos_ttl_graph, format="turtle")



<Graph identifier=N2811bce500da4e99b66633aded1fd0c9 (<class 'rdflib.graph.Graph'>)>